# GenWorlds First Steps Tutorial

Welcome to GenWorlds! In this beginner's tutorial, we'll walk through creating a simple GenWorld from scratch. By the end, you'll have a basic understanding of how GenWorlds operates, setting the stage for more advanced use cases.

## Scenario Overview:

Imagine a world where two agents, "John" and "Matthew", work in tandem. John receives a request from a user to generate a random matrix and determine its determinant. John then uses a tool to craft this matrix, which he sends over to Matthew. Matthew, equipped with a different tool, calculates the determinant of the matrix John created. After doing so, Matthew sends back the result to John, who then delivers the original matrix and the determinant value to the user.

While this might sound like overkill for a simple task (since the LLM could probably do this in one go), the purpose is to demonstrate how different components of GenWorlds can interact.

## Step 1: Initial Set Up

Before diving into our world, we need a few basic utilities and configurations.

In [ ]:
from datetime import datetime
import threading
from typing import List
from time import sleep
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Step 2: Crafting the World

Our world, named "Compute Matrix Determinant World", will initially be empty. We'll then populate it with agents and objects in subsequent steps.

In [ ]:
from genworlds.worlds.concrete.base.world import BaseWorld

# Define the World
CMD_world = BaseWorld(
    name="Compute Matrix Determinant World",
    description="A world where two agents interact to generate a matrix and compute its determinant.",
)
CMD_world.launch()

## Step 4: Introducing the Matrix Generator

To generate matrices, we'll use an object called `MatrixGenerator`. This object will listen to agents' requests to craft matrices and respond accordingly.

In [ ]:
from genworlds.objects.abstracts.object import AbstractObject
from genworlds.events.abstracts.event import AbstractEvent
from genworlds.events.abstracts.action import AbstractAction

# Create two events and one action
# Event that the agent will use to generate a matrix
class AgentGeneratesNxNMatrixEvent(AbstractEvent):
    event_type = "agent_generates_n_by_n_matrix_event"
    description = "An agent generates a squared matrix of size N of integer numbers"
    N: int

# Event that the MatrixGenerator Object will use to give the requested matrix to the agent
class SendGeneratedMatrixEvent(AbstractEvent):
    event_type = "send_generated_matrix_event"
    description = "Sends the requested squared matrix of size N of integer numbers to the agent"
    matrix: List[List[int]]
        
class GenerateSquaredMatrix(AbstractAction):
    trigger_event_class = AgentGeneratesNxNMatrixEvent
    description = "Generates squared matrices of size N."
    
    def __init__(self, host_object: AbstractObject):
        self.host_object = host_object
    
    def __call__(self, event:AgentGeneratesNxNMatrixEvent):
        import numpy as np
        N = event.N
        matrix = np.random.randint(100, size=(N, N))
        event = SendGeneratedMatrixEvent(
            sender_id=self.host_object.id,
            target_id=event.sender_id,
            matrix = matrix.tolist(),
        
        )
        self.host_object.send_event(event)

# Define the MatrixGenerator Object
class MatrixGenerator(AbstractObject):
    def __init__(self, id:str):
        actions = [GenerateSquaredMatrix(host_object=self)]
        super().__init__(name="Matrix Generator", 
                         id=id, 
                         description="Object used to random integer squared matrices.", 
                         actions=actions
                         )

# Instantiate the MatrixGenerator Object
matrix_generator = MatrixGenerator(id="matrix_generator")

# Incorporate the Matrix Generator into the Simulation
CMD_world.add_object(matrix_generator)

## Testing without Agents

In [ ]:
from genworlds.utils.test_user import TestUser

# Create a Testing User
test_user = TestUser()

In [ ]:
message_to_send = AgentGeneratesNxNMatrixEvent(
    sender_id=test_user.id,
    target_id="matrix_generator",
    N=10
).json()

test_user.socket_client.send_message(message_to_send)

## Step 5: Creating a Dummy Agent for Simulation

In this step, we will be creating a simple dummy agent for our simulation using the genworlds library. The agent will have a basic set of goals, a role, and some thought processes attached to it. The main idea behind this step is to understand how to set up an agent with basic functionalities and then integrate it into a given simulation environment.

### Overview:

1. Importing Necessary Modules: Begin by importing the required classes and modules from genworlds.

2. Function for Agent Generation: We will define a reusable function named generate_dummy_agent that will generate an agent based on a given name. The function will configure the agent's role, background, personality, goals, constraints, evaluation principles, and topic of conversation.

3. Agent's Brains: The function will then set up the agent's 'brains' using the NavigationThought and EventFillerThought classes. These 'brains' dictate how the agent thinks and operates in the simulation.

    - Navigation Brain: Responsible for helping the agent navigate through the simulation.

    - Execution Brains: Defines the agent's execution strategies, such as filling in events in this case.

4. Agent Action Brain Map: Defines the default action strategy for the agent.

5. Agent Generation: Using our function, we'll generate an agent named "John" and integrate him into our simulation using CMD_simulation.register_agent.

In [ ]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

agent_name = "John"
description = """Agent that helps the user generate random matrices. Can talk to other agents to ask for information."""

# Generate a Dummy Agent named John
john = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

john.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
john.add_wakeup_event(event_class=AgentSpeaksWithAgentEvent)

## Attach John to the Simulation
CMD_world.add_agent(john)

## Step 6: Simulating User Interaction with the Agent

In this step, we'll demonstrate how to simulate user interaction with the agent in the simulation environment. The objective is to have a pseudo-user send a request to our dummy agent "John", asking him to perform specific tasks. This helps in understanding the dynamics of agent-user communication and to observe how the agent reacts and processes user's requests.

In [ ]:
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

# Format the message that will be sent to the simulation socket
test_msg = "Hey John, generate a 4x4 matrix and send it to me please!"
message_to_send = UserSpeaksWithAgentEvent(
    sender_id=test_user.id,
    message=test_msg, 
    target_id="John"
).json()

# Send the message to John
test_user.socket_client.send_message(message_to_send)

## Step 7: Integrating a Determinant Calculator into the Simulation

In this step, we will introduce an object into our simulation environment that agents can utilize to compute the determinant of matrices. This serves as a representation of how tools and utilities can be made available to agents, enhancing their capabilities.

In [ ]:
from genworlds.objects.abstracts.object import AbstractObject
from genworlds.events.abstracts.event import AbstractEvent
from genworlds.events.abstracts.action import AbstractAction

# Event that the agent will use to compute the determinant of a given matrix
class AgentComputesDeterminant(AbstractEvent):
    event_type = "agent_computes_determinant"
    description = "An agent computes the determinant of a matrix"
    matrix: List[List[int]]

# Event that the DetCalculator Object will use to give the requested determinant to the agent
class SendMatrixDeterminant(AbstractEvent):
    event_type = "send_matrix_determinant"
    description = "Sends the requested determinant of the matrix to the agent"
    determinant: int
        
class ComputeDeterminant(AbstractAction):
    trigger_event_class = AgentGeneratesNxNMatrixEvent
    description = "Generates squared matrices of size N."
    
    def __init__(self, host_object: AbstractObject):
        self.host_object = host_object
    
    def __call__(self, event: AgentComputesDeterminant):
        import numpy as np
        determinant = np.linalg.det(np.array(event.matrix))
        event = SendMatrixDeterminant(
            sender_id=self.host_object.id,
            target_id=event.sender_id,
            determinant = determinant,
        )
        self.host_object.send_event(event)

# Define the DetCalculator Object
class DetCalculator(AbstractObject):
    def __init__(self, id:str):
        actions = [ComputeDeterminant(host_object=self)]
        super().__init__(name="Determinant Calculator", 
                         id=id, 
                         description="Object used to compute determinants of squared matrices.", 
                         actions=actions
                         )

# Instantiate the DetCalculator Object
det_calculator = DetCalculator(id="det_calculator")

# Incorporate the Determinant Calculator into the World
CMD_world.add_object(det_calculator)

## Step 8: Introducing Another Agent and Requesting World State Updates
In this step, we will be adding a second agent, "Matthew", to our simulation. Both John and Matthew will then request updates on the world state, allowing them to become aware of each other and other changes in the simulation environment.

In [ ]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

agent_name = "Matthew"
description = """Agent that helps to compute determinants of matrices. Can talk to other agents to ask for information."""

# Generate a Dummy Agent named John
matthew = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

matthew.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
matthew.add_wakeup_event(event_class=AgentSpeaksWithAgentEvent)


## Attach DCPI to the Simulation
CMD_world.add_agent(matthew)

## Step 9: Final Collaborative Test between Agents

In the final step of this tutorial, we're putting everything together to demonstrate how agents can collaborate in the simulation world. We'll instruct John to generate a 3x3 matrix, have its determinant computed by Matthew, and then relay the matrix and its determinant back to us, the user.

In [ ]:
test_msg = """Hey John, 
generate a matrix 3x3, 
send it to Matthew to compute its determinant, 
and when he replies back to you, tell me the matrix and its determinant."""

message_to_send = UserSpeaksWithAgentEvent(
    sender_id=test_user.id,
    message=test_msg, 
    target_id="John"
).json()

# Send the message to John
test_user.socket_client.send_message(message_to_send)

## Conclusions

Congratulations on reaching the end of this tutorial on simulating agent collaboration in the GenWorlds environment! Let's reflect on what we've accomplished and what we've learned:

### Key Takeaways:
1. **Basics of the GenWorlds Framework:**
    - We began with a solid foundation, introducing the core components and functionalities of the GenWorlds simulation framework. This framework is powerful and flexible, enabling the creation and management of complex agent-driven worlds.
2. **Creation of Dummy Agents:**
    - We went through the process of creating basic, dummy agents and setting up their attributes. This gave us a hands-on understanding of agent attributes like name, role, background, and their thought processes like navigation_brain.
3. **User-Agent Interaction:**
    - We designed a Fake User to simulate real-world interactions between users and agents in the environment. This user-agent interaction plays a crucial role in instructing agents and receiving feedback from them.
4. **Defining Custom Events & Objects:**
    - Our tutorial introduced the concept of custom events, allowing us to define specific actions like matrix determinant computation. We also learned about objects in the simulation, such as our "Determinant Calculator."
5. **Inter-Agent Collaboration:**
    - One of the highlights was demonstrating how agents can collaborate. We instructed John to create a matrix, which was then processed by Matthew, showcasing the potential for multi-agent workflows.

### Further Exploration:
While we've covered a lot, the GenWorlds environment offers even more to explore:
- **Complex Agent Behaviors:** Our tutorial focused on basic, dummy agents. Delve deeper into the framework to create agents with intricate behaviors, decision-making processes, and reactions to diverse events.
- **Expand the World:** Introduce more objects, locations, and complexities to your simulation. Imagine scenarios like agents navigating through a maze or collaborating to solve puzzles.
- **Real-time Analytics:** With multiple agents and complex events, the simulation can produce vast amounts of data. Dive into analyzing this data to gain insights into agent behaviors, interactions, and more.

### Wrapping Up:
Agent-based simulations like GenWorlds are powerful tools for modeling complex systems and interactions. They offer insights into multi-agent behaviors, decision-making processes, and collaborative efforts. By mastering these simulations, you'll be equipped to tackle intricate problems, design intelligent systems, and model real-world scenarios with confidence.

Thank you for journeying through this tutorial with us. We hope it has sparked your interest in exploring the vast possibilities of agent-based simulations further!